<center>
<img src="https://raw.githubusercontent.com/FUlyankin/ekenam_grand_research/master/images/cover.png">
</center>


# <center> Иканам гранд рисёрч </center>
## <center>  Туториал: Как скачать данные из контакта </center> 


Проект **Иканам гранд рисёрч** реализуется [Иканам стьюдентс коммьюнити,](https://vk.com/ikanam)
в частности [вот этим парнем по имени Филипп.](https://vk.com/ppilif) Если вы нашли ошибку или у вас есть предложения, замечания, деньги, слава или женщины, можно ему написать. Весь говнокод, использованный в исследовании распространяется по лицензии [Creative Commons CC BY-NC-SA 4.0.](https://creativecommons.org/licenses/by-nc-sa/4.0/) Его можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала. При наличии технической возможности необходимо также указать активную гиперссылку на [страницу рисёрча.](https://github.com/FUlyankin/ekenam_grand_research)

## 1. Подготовка

Многие сайты и сервисы хотят упростить жизнь программистам и предоставляют им в пользование уже готовые куски кода, к которым можно просто обратиться и получить нужную информацию о сервисе. Такие куски кода называются API (Application programming interface). Вконтакте не является исключением.

Сразу же стоит сказать, что API Вконтакте это рай для дата-сайентиста. Можно скачать огромное количество всего без серьёзных ограничений. Например, Facebook по своему API отдаст вам информацию только о тех людях, которые установили ваше приложение, API Одноклассников потребует написать им письмо с объяснением того, зачем это вам понадобились их данные, а API Вконтакте отдаст вам всё практически задаром. Единственное, что он попросит — это не обращаться к сайту очень часто, и это очень круто.

Тем не менее, недавно Вконтакте провёл серьёзную реформу своей музыкальной части и монетизировал её. Это привело к изъятию куска API, связанного с музыкой из открытого доступа. Более того, существует опасность, что социальная сеть продолжит резать бесплатный функционал своего API после разных скандалов, связанных с банками, нагло собирающими информацию о пользователях для своих скоринговых моделей. В связи с этим, нужно успеть нарисёрчить по данным из соц сеток вдоволь, пока нас существенно не ограничили в их выгрузке.

Ознакомиться со способами взаимодействия с Вконтакте можно [по ссылке, о существовании которой вы даже не догадывались...](https://vk.com/dev/manuals) По ней можно найти лучшего друга человека —  понятную документацию. Для того, чтобы получить доступ к выгрузке данных, нужно пройти ряд бюрократических процедур.

Первая такая процедура заключается в [создании своего приложения.](http://vk.com/editapp?act=create) Заполняем все поля, загружаем предварительно подготовленные картинки нужного размера. Разрешаем доступ этому приложению к своей странице.

<img align="center" src="https://raw.githubusercontent.com/FUlyankin/vk_grand_research/master/images/vk6.png" height="500" width="500">

<img align="center" src="https://raw.githubusercontent.com/FUlyankin/vk_grand_research/master/images/vk_2.png" height="600" width="600">

Для работы с частью методов API этого вполне достаточно (обычно в заголовке такого метода стоит соответствующая пометка). Для части методов, используемых нами, может понадобиться ключ доступа.

Для того, чтобы получить его, необходимо сделать ещё пару странных манипуляций:

Переходим по ссылке вида (на месте звездочек должен стоять ID созданного вами приложения):

> https://oauth.vk.com/authorize?client_id=**********&scope=14&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.16&response_type=token


В итоге по этому запросу будет сформирована ссылка следующего вида:

> https://oauth.vk.com/blank.html#access_token=95b636116ef20e0798fe4dff380511fc13&expires_in=86400&user_id=6045249


Первый набор знаков - access token, т.е. маркер доступа. Вторая цифра (expires_in=) время работы маркера доступа в секундах (одни сутки). По истечению суток нужно будет получить новый маркер доступа. Последняя цифра (user_id=) ваш ID Вконтакте. Нам в дальнейшем понадобится маркер доступа и ID.  Для удобства сохраним их.

Обратите внимание на ссылку, по которой мы делали запрос на предоставление токена. Внутри неё находится странный параметр `scope=14`. Эта загадочная цифра ничто иное, как права доступа к социальной сети. Подробнее [познакомиться с взаимно-однозначным соответствием между числами и правами можно в документации.](https://vk.com/dev/permissions) Например, если мы хотим получить доступ к друзьям, фото и аудио, мы подставим в `scope` цифру $2 + 4 + 8 = 14$.  


In [ ]:
token= "74ff27e52b6aa780fc2f3be6c5f0b5a87d0709a071fb93791e33f3971e7c8e23e3d9e767db787d1db2866"
myid = "6045249"

## 2. Учимся собирательству

Для начала подгрузим парочку пакетов. 

In [ ]:
import requests                  # Пакет для скачки данных из этих ваших интернетов
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами
import matplotlib.pyplot as plt  # Пакет для строительства графиков
import time            # Пакет для работы со временем. Например, помогает ставить заглушки 
                       # time.sleep(секунды), необходимые для того что ВК не банил нащего
                       # сборщика данных из-за слишком частых запросов
        
# Пакет для красивых циклов. При желании его можно отключить. Тогда из всех циклов придётся 
# удалять команду tqdm_notebook.
from tqdm import tqdm_notebook   # подробнее: https://github.com/tqdm/tqdm

Чтобы задать вопрос API вконтакте, нужно сформировать ссылку следующего вида:

> https://api.vk.com/method/METHOD_NAME?PARAMETERS&access_token=ACCESS_TOKEN

> https://название_api/используемый_метод?параметры_метода&другие_опции&ключ_доступа

В ссылку сначала встраивается метод, с помощью которого мы осуществляем запрос и какие-то параметры для этого метода. Список всех существующих методов и параметров можно изучить, конечно же, в [документации.](https://vk.com/dev/methods)

Попробуем узнать своё имя. Метод `users.get`  возвращает расширенную информацию о пользователе. Параметр `user_ids` отвечает за `id` пользователей, информацию о которых нам хотелось бы вытащить.

In [ ]:
url = "https://api.vk.com/method/users.get?user_ids=6045249&access_token="+str(token)

In [ ]:
response = requests.get(url) 
response

Ура! Благословенный 200-ответ! Мы получили его в удобном json формате, который python воспринимает как словарик.

In [ ]:
my_name = response.json()
my_name

Со словарями очень удобно работать. Круто, когда можно узнать как тебя зовут с комфортом. 

In [ ]:
infa = my_name['response'][0]
infa

Оформим всё, что мы сделали выше в виде удобной функции, которая будет пригодна для любых методов и параметров.

In [ ]:
def vk_download(method, parameters, token = token):
    url = 'https://api.vk.com/method/'+method+'?'+parameters+'&access_token='+str(token)
    response = requests.get(url) 
    return(response.json())

vk_download('users.get','user_ids=6045249')

В принципе это всё. Дальнейшая стратегия будет такой: открываем документацию, находим нужные метод и параметры и применяем их. Попробуем с помощью новых знаний решить несколько рисёрческих задачек. Начнём с психологии. 

Все цитаты в пабликах вконтакте делятся на два вида: мудрые и недостаточно мудрые. Если цитата мудрая, то её не грех репостнуть на стеночку. Поговаривают, что если в цитате есть личные местоимения, то пользователь автоматически сопоставляет себя автору этой цитаты и вероятность того, что она буде репостнута повышается. Например, цитату 

> Можно простить человеку все, кроме отсутствия. (Экзюпери) 

мне не очень хотелось бы репостить. А вот цитата 

> Мне наплевать, что вы обо мне думаете. Я о вас вообще не думаю. (Булгаков) 

совершенно другое дело. 


Гипотеза сформулирована. Глаза загорелись. Пришло время проверить её. 

## 3. Какая мудрость более мудрая 

Для начала выгрузим пару записей со своей стены, чтобы ещё раз осознать как это всё работает. В качестве ответа на запрос мы получим длинный страшный кусок текста. В этом куске текста будет абсолютно все, что есть у нас на стене. Ссылки на картинки, количество лайков под каждой записью, подробная история того как и откуда был сделан репост записи и так далее. Нас среди всего этого добра будут интересовать только лайки, репосты и просмотры. 

In [ ]:
wall = vk_download('wall.get','user_ids=6045249')['response']
print(wall[0])
print(wall[1])

In [ ]:
len(wall)

Скачалось целых 20 записей. Первая запись - число постов на стене. При этом в графе `likes` лежит часть того, что мы жаждем. 

In [ ]:
wall[3]['likes']

Вытащим все лайки со стены. 

In [ ]:
[item['likes']['count'] for item in wall[1:]]

Отлично! Мы готовы сделать первый рисёрчерский вывод. Кажется, обладатель этой стены среднепопулярный парень... 

Теперь попробуем провернуть аналогичный трюк с лайками из какой-нибудь [мудрой группы.](https://vk.com/ooioioioioioo) Отметим, что при этом есть одна тонкость, которая прописана в [описании метода wall.get.](https://vk.com/dev/wall.get) Если мы хотим вытащить записи из группы, то ее id должен быть записан со знаком минус. Сразу же обратим внимание на то, что нам хотелось бы получить больше $20$ записей. Пороемся в документации. Выясняем, что существуют такие параметры как `count` - число записей для скачки (не может превышать 100) и `offset` - смещение, необходимое для выборки определённого подмножества записей. 

In [ ]:
[item['likes']['count'] for item in vk_download('wall.get','owner_id=-26610299&count=5')['response'][1:]]

In [ ]:
[item['likes']['count'] for item in vk_download('wall.get','owner_id=-26610299&count=5&offset=2')['response'][1:]]

Отлично! Мы только что скачали со стены первые пять лайков, а затем ещё пять со сдвигом два. Обратите внимание, что вторые три элемента первого списка и первые три второго совпадают.

Дело осталось мизерное, написать цикл, который пройдёт по всей стенке и соберёт с неё лайки, тексты постов и репосты. 

In [ ]:
# В этот словарик будем собирать информацию о лайках, репостах и текстах
wall_inf = {'likes' : [ ],
            'reposts' : [ ],
            'text' : [ ]}

# Выясним сколько всего в группе постов 
n = vk_download('wall.get','owner_id=-26610299')['response'][0] 
print('В группе целых', n, 'мудростей')

# Цикл обёрнут в команду-вызыватель загрузочной строки. Можно её удалить.
# Возможно, для её корректной работы понадобится виджеь, который можно 
# скачать вот тут: https://ipywidgets.readthedocs.io/en/stable/user_install.html

for i in tqdm_notebook(range(0, n + 100, 100)): 
    # После каждого запроса будем ждать 0.4 секунды, иначе вк не отдаст данные. 
    # Чуть подробнее про заглушки мы ещё раз поговорим ниже. 
    time.sleep(0.4)
    # Взяли себе очередные 100 записей со стены (не забываем что нулевая запись - длина стены)
    # Именно поэтому мы её обрезаем. 
    wall = vk_download('wall.get','owner_id=-26610299&count=100&offset='+str(i))['response'][1:]
    # Вытащили лайки 
    new_likes = [item['likes']['count'] for item in wall]
    # Вытащили репосты 
    new_reposts = [item['reposts']['count'] for item in wall]
    # Вытащили тексты постов 
    new_text = [item['text'] for item in wall]
    # Закинули всё это добро в словарик с данными 
    wall_inf['likes'].extend(new_likes)
    wall_inf['reposts'].extend(new_reposts)
    wall_inf['text'].extend(new_text)

Обратите внимание, что для сообществ нельзя делать более 20 запросов в секунду! Про все существующие ограничения можно прочесть всё в той же самой [документации.](https://vk.com/dev/api_requests?f=3.1.%20Частотные%20ограничения) В наших руках тем временем оказалось очень много мудрости. Превратим всю нашу мудрость в таблицу.

In [ ]:
df_wise = pd.DataFrame(wall_inf)
df_wise.head()

Возьмём все наши мудрые цитаты, уменьшим в них литеры методом `.lower()`, а после разобьём их по пробелам на слова методом `.split(' ')`. После мы сможем проверить есть ли в цитате местоимение "я".

In [ ]:
df_wise.text[2].lower().split(' ')

In [ ]:
df_wise['I'] = ['я' in post.lower().split(' ') for post in df_wise.text]
df_wise['I'].sum( ) 

Отлично! Местоимение "я" содержится в довольно большой части выборки. Наблюдений для проверки гипотезы нам очень даже хватает. Посчитаем среднее число лайков и среднее число репостов для постов обоих типов. 

In [ ]:
df_wise[df_wise['I'] == 1].mean(axis=0)

In [ ]:
df_wise[df_wise['I'] == 0].mean(axis=0)

In [ ]:
# Картиночки
plt.figure(figsize=(15, 4))
plt.subplot(1,2,1)
df_wise[df_wise['I'] == 1]['likes'].plot.hist(title='Цитаты, с местоимением')

plt.subplot(1,2,2)
df_wise[df_wise['I'] == 0]['likes'].plot.hist(title='Цитаты без местоимения')

plt.show()

Хмм... Под постами без этого чудного местоимения лайков и репостов, в среднем, поменьше. Проверим значимо ли это различие для лайков. Репосты останутся как домашнее задание.  

$H_0\colon$ в среднем посты лайкают одинаково. 

$H_1\colon$ не одинаково.

О боже! Это же гипотеза о равенстве средних значений двух произвольно распределённых случайных величин. Вычислим значение $t$-статистики, которая будет иметь асимптотически нормальное распределение и сравним её значение с критическим. Будем при этом считать, что дисперсии у наших случайных величин, разные. 

In [ ]:
# Модуль для првоерки гипотез 
import scipy.stats

likes_I = np.array(df_wise[df_wise['I'] == 1]['likes'])
likes_noI = np.array(df_wise[df_wise['I'] == 0]['likes'])

scipy.stats.ttest_ind(likes_I , likes_noI, equal_var = False)

Гипотеза о равенстве средних для данного паблика отвергается на 5% уровне значимости. Вы владелец этого паблика? Постите побольше цитат с местоимением "Я" и успех к вам придёт (но это неточно).

Пришло время сделать что-нибудь банальное. Настолько банальное, что читатель скажет "Да вы лрпкрпзшпгйп3рхйп3ршхпх, это же уже было и я рпкшрп3ршгпкшгкйпщошцуждауш вас всех!!!" 

## 4. Друзья и графы

### 4.1 Выкачиваем друзей 

Скачаем с помощью метода [`friends.get`](https://vk.com/dev/friends.get) всех своих друзей, а после построим их граф.  

In [ ]:
friends_ids = vk_download('friends.get','user_ids=6045249')
friends_ids = friends_ids['response']
friends_ids[:5]

Вот они, все мои друзья! 

In [ ]:
len(friends_ids)

На самом деле метод `friends.get` предусматривает огромное количество параметров, среди них есть параметр `fields`, который отвечает за то какую именно информацию о друзьях нужно вернуть. Спектр этой информации довольно широкий. От пола до университета. 

Если этот параметр никак не задействовать, то функция в качестве ответа выдаст список ID. Если его задействовать в совершенно любом виде, то в добавок к дополнительной информации появятся еще и имена. Нам для построения графа достаточно будет знать только ID всех друзей. Однако можно сделать граф более красивым. С именами, а может быть даже фотографиями. 

Для построения графа нам понадобится метод `friends.getMutual`, который возвращает список общих друзей с каким-нибудь человеком. 

In [ ]:
# Например, вот столько общех друзей у меня с юзером 10592581
len(vk_download('friends.getMutual','target_uid=10592581')['response'])

Вытаскиваем общих друзей для каждого из своих друзей и записываем в словарик, по которому мы впоследствии будем строить наш граф.

In [ ]:
infa = { }
for friend in tqdm_notebook(friends_ids):
    current = vk_download('friends.getMutual','target_uid='+str(friend))
    infa[friend] = current   

Изучим поподробнее информацию, которую мы скачали и увидим пару странностей, а именно: 

```   
382202404: {'error': {'error_code': 6,
   'error_msg': 'Too many requests per second',
   'request_params': [{'key': 'oauth', 'value': '1'},
    {'key': 'method', 'value': 'friends.getMutual'},
    {'key': 'target_uid', 'value': '382202404'}]}}

```

Откуда берутся такие друзья? На любой респонс всегда найдётся свой реквест! Посетим страничку такого друга...
Это вполне себе живой друг, информация для которого не выкачалась. Почему? Потому что, как уже упоминалось, есть ограничения, и мы в секунду сделали слишком много запросов... 

Вот беда... Код слишком быстрый. Нужно научить его ждать. Сделать это нам поможет пакет `time`. 

<img align="center" src="https://raw.githubusercontent.com/FUlyankin/vk_grand_research/master/images/waiter.jpg" height="450" width="450"> 


Вроде бы Вконтакте позволяет делать не более 3 запросов в секунду. Поэтому будем выжидать после каждой итерации $0.4$ секунды, чтобы не пробить лимиты. 

In [ ]:
k = 0  # счётчки для пересчёта плохих друзей
infa = { }
for friend in tqdm_notebook(friends_ids):
    time.sleep(0.4)  # Спать!  
    try:
        infa[friend] = vk_download('friends.getMutual','target_uid='+str(friend))['response']
    # Если юзер выпилил себя из контакта, то код выше выдаст ошибку, т.к. для такого пользователя
    # не будет респонса, а будет ошибка. Подсчитаем количество таких пользователей. 
    except:
        infa[friend] = [ ]
        k = k + 1
print('Количество пользователей, удаливших свою страничку: ', k)

Итак, количество плохих друзей в нашем списке уменьшилось. Посмотрим кто там остался. 

```
1310129: {'error': {'error_code': 15,
   'error_msg': 'Access denied: user deactivated',
   'request_params': [{'key': 'oauth', 'value': '1'},
    {'key': 'method', 'value': 'friends.getMutual'},
    {'key': 'target_uid', 'value': '1310129'}]}},
```

Ага! Деактивэйтед юзерс. Или просто люди, которые удалили свою страницу. 



<img align="center" src="images/vk_del.jpg" height="450" width="450">


Пока работал этот кусок кода, я успел пересчитать всех таких пользователей. Всё сходится! Друзей скачали. Пришло время рисовать граф. 


###  4.2 Networkx 

Попробуем немного проанализировать граф своих друзей. В анализе графа нм поможет библиотека `networkx`. Если у вас её нет, наверное, есть смысл её установить. 

``` pip3 install networkx```

В этой библиотеке реализовано довольно [много крутых алгоритмов](https://networkx.github.io/documentation/networkx-1.9/reference/algorithms.html) на графах. Более того, по ним сделана довольно красивая [документация.](https://networkx.github.io/documentation/stable/) 

Также мы будем использовать в ходе нашего анализа пакет `nxviz`. Его также не помешает установить.

``` pip3 install nxviz```

Итак, создаём граф и рисуем наши первые картинки! 

In [ ]:
import networkx as nx             # Пакет для работы с графами
import nxviz as nv                # Немного визуализаций 

import matplotlib.pyplot as plt   # Пакет для визуализации
plt.style.use('ggplot')           # Правильный стиль для отрисовки картинок
%matplotlib inline               

Прошу любить и жаловать! Граф друзей.

In [ ]:
plt.figure(figsize=(8,8))

FG = nx.from_dict_of_lists(infa)
nx.draw(FG, node_size=30,  linewidths=0.05, cmap=True)

Нарисуем матрицу связности для нашего графа. Темные точки означают наличие рёбер. 

In [ ]:
m = nv.MatrixPlot(FG)
m.draw()

Пакет `NetworkX` даёт довольно тошнотворные визуализации. Конечно же можно как следует поднапрячься, засесть на пару суток и родить граф-конфетку внутренними средствами пакета. Однако мы пойдём другой дорогой.

Если как следует погуглить, то можно найти [вот такое вот офигенное приложение](http://www.yasiv.com/vk) для контакта, которое строит граф друзей. Только дай ему доступ и на выход ты получишь вот такую офигительную интерактивную, быстро работающую штуку. 

<img align="center" src="images/2. loises_and_graph/d3_fr_gr.png" width="900"> 

<br>

Можно добавлять людей, раскрашивать рёбра в другие цвета и всё такое. Можно даже выкинуть какого-нибудь человека и посмотреть на то как граф разваливается на части, и освободившиеся вершины разлетаются в разные стороны. Эта визуализация богоподобна, а её автор гений. Мой горький опыт говорит мне о том, что каждая хорошая визуализация это великое страдание. Чтобы построить такую прекрасную визуализацию, автор страдал довольно долго. Он писал на java script для визуализации [свою библиотеку.](https://github.com/anvaka/VivaGraphJS) На хабре можно найти [его статью](https://habrahabr.ru/post/144758/) об этом.

Мы попробуем родить нечто похожее. К моему превеликому сожалению, я не могу в javascript, поэтому мы будем рожать нечто похожее по уже готовым шаблонам. Из статьи выше можно узнать, что основным тормозом подобных визуализаций является браузерный код прорисовки элементов. Автору удалось перетащить это бутылочное горлышко в свой алгоритм укладки. Благодаря этому его визуализация работает быстро. Это плохая новость для нас. Наш шаблон будет хорош только для маленьких графов. Большие будут сильно тормозить.

В сети выложено довольно много [шаблонов с красивым интерактивом.](https://github.com/d3/d3/wiki/Gallery) Люди, которые выкладывают эти визуализации в сеть - прекрасны. Они делают нас на шаг ближе к дзену, ровно, как и команда `import this`. 

Нас будет интересовать [шаблон графа.](https://bl.ocks.org/heybignick/3faf257bbbbc7743bb72310d03b86ee8) В конечном счёте, мы сможем открывать его в отдельной вкладке и дёргать за вершины. Возьмём шаблон и разберёмся в том как его правильно использовать. На страничке с описанием графа есть две вставки. Одна из них называется index.html 

<img align="center" src="images/2. loises_and_graph/d3_index.png" width="550"> 

В этой вставке задаётся внешний вид графа и его моторика. Если вы знаете javascript или просто (как я) обладаете хорошей интуицией и можете в метод тыка и гугл, вы можете видоизменить дизайн графа.  Вторая вставка называется miserables.json.

<img align="center" src="images/2. loises_and_graph/miserables.png"  width="550"> 

В этой вставке находится содержимое графа. В виде словарика задаётся какие вершины с какими соединяются и как подписываются. Эти два куска кода можно скопировать к себе на рабочий стол в два разных текстовых файла. 

<img align="center" src="images/2. loises_and_graph/d3_desktop.png" width="300"> 

Теперь, если открыть файл index.html в браузере firefox, вашему взору предстанет та самая визуализация с сайта. Наша задача заключается в следующем: мы должны сгенерировать словарик со своими друзьями и связями между ними, а после закинуть его в файл miserables.json. Таким образом, интерактивная визуализация будет готова и мы даже сможем поделиться ей со своими друзьями.

Итак, сгенерируем словарик следущего вида: 

```
{ 
"nodes": [ 
        {"id":"Aлександр Конев", "group":1}, 
        {"id":"Александр Тишин", "group":1}, 
        {"id":"Давыдкина Кристина","group":1} 
        ], 
"links":[ 
        {"source":"Aлександр Конев", "target":"Александр Тишин", "value":1}, 
        {"source":"Aлександр Конев", "target":"Давыдкина Кристина", "value":8} 
        ] 
}
```

Первый список - вершины. Цифра в `group` отвечает за цвет вершины. Второй список - рёбра. `source` и `target` означают вершины, которые нужно соединить ребром. Цифра в `value`, судя по всему, отвечает за толщину ребра. Построить такой json-словарик нам поможет замечательный файлике `ikanam_info`, который выглядит следующим образом и преследует нас уже довольно давно. 

Выкачиваем по каждому другу информацию о имени, фамилии и полу.

In [ ]:
res = vk_download('users.get','user_ids='+str(11105)+'&fields=sex')
res

In [ ]:
# Имя 
res['response'][0]['last_name'] + ' ' + res['response'][0]['first_name']

In [ ]:
# Пол 
res['response'][0]['sex']

У нас уже есть словарик `infa`, где по ключу `id` хранится список из друзей, с которыми есть связь. Создадим ещё один словарик `id_names`, где по ключу `id` будет храниться базовая информация про юзера (фотка, имя , пол).

In [ ]:
id_names = { } # В этом словарике будем хранить информацию про юзера

for fr in tqdm_notebook(friends_ids):
    time.sleep(0.4)   # Не забываем отдохнуть между запросами
    try:
        cur_el = { }      # Информация по текущему другу будет храниться тут 
        res = vk_download('users.get','user_ids='+str(fr)+'&fields=sex,photo')['response'][0]
        cur_el["id"] = res['last_name'] + ' ' + res['first_name']
        cur_el["group"] = res['sex']
        # Закидываем нового человека в словарик 
        id_names[fr] = cur_el        
    except:
        # Если скачка не удалась, выведем сообщение об ошибке
        print('Ошибка с ', fr)

In [ ]:
id_names[12846897]

Отлично! Теперь на основе словариков `id_names` и `infa` мы модем сделать необходимый нам .json файлик. 

In [ ]:
nodes = [ ]  # Тут будем хранить вершины
links = [ ]  # Тут будем хранить рёбра 
for fr in infa.keys():
    nodes.append(id_names[fr]) # создали для текущего юзера вершину 
    # Создаём для текущей вершины все возможные рёбра
    for fr_my_fr in infa[fr]:
        cur_edge = { }
        cur_edge["source"] = id_names[fr]['id']
        cur_edge["target"] = id_names[fr_my_fr]['id']
        cur_edge["value"] = 1 # Этот параметр ни на что не влияет
        links.append(cur_edge)
        
print(nodes[:5])
print('\n')
print(links[:5])

Посмотрим сколько в итоговом графе окажется рёбер, а затем созраним наш json. 

In [ ]:
len(links)

In [ ]:
import json

itog_js = { "nodes": nodes, "links": links }
with open("miserables.json","w") as f:
    json.dump(itog_js, f)

Всё готово! Помещаем нвоый файлик рядом с html, открываем её и видим граф своих друзей. При большом количестве друзей он строится довольно медленно и анимация подтормаживает. Тем не менее, маленькие графы, выглядят довольно красиво. 

<img align="center" src="images/2. loises_and_graph/d3_13_gr.png"  width="850"> 


## 5. Лайкограмма 

Граф друзей это конечно же хорошо, но никак не отражает реального положения дел. Намного интереснее было бы посмотреть на лайки. Если люди реально общаются, то они лайкают друг-друга. 

Более того, многие из нас когда-то оказывались в подобных ситуациях: 

<img align="center" src="http://memesmix.net/media/created/bmflc0.jpg" height="350" width="350"> 

Возможно, вы подумаете, что у нас есть решение этой проблемы. На самом деле нет. Быть плохим примером гораздо веселее. Мы будем играть на другой стороне, на стороне стервозных сучек. Добро пожаловать подкат, милая. Современный мир огромными шагами идёт в сторону автоматизации рутинных процессов. Мы поможем тебе автоматизировать подготовку к твоим бессмысленным истерикам и дадим тебе автоискатель почвы для твоего врождённого измухослонаделателя. Возьмём всех своих друзей и попробуем посмотреть как внутри неё происходит циркуляция лайков. 

### 5.1 Подготовка

Напишем несколько функций. Первая функция будет собирать тех, кто лайкнул стену нашего пользователя. В этом нам поможет уже знакомый нам метод `wall.get`. Напомним, что в лучшем друге человека, документации, написано, что у этого метода есть параметры `count` - число записей для скачки (не может превышать 100) и `offset` - смещение, необходимое для выборки определённого подмножества записей.  Ничего нового, вроде бы, в функции ниже нет. 

In [ ]:
def Get_Wall(this_id):
    # Делай раз! Выясним сколько у пользователя на стене записей...
    n = vk_download('wall.get','owner_id=' + str(this_id))['response'][0]
    
    # Качаем со стены все записи!
    chto_on_postit = [ ]
    for i in range(0,100*(n//100+1),100):
        # Качаем внеочередные 100 записей со стены.
        time.sleep(0.4) # Не забываем немного вздремнуть 
        # Первый элемент всегда число записей, игнорируем его 
        wall = vk_download('wall.get', 'owner_id=' + str(this_id) +'&count=100&offset='+str(i))['response'][1:]
        chto_on_postit.extend(wall)
        
    # В файлике chto_on postit будут лежать грязные стенки.
    return chto_on_postit

In [ ]:
dw = Get_Wall(friends_ids[0])
dw[0]

Отлично! Грязная стена выгружается. Попробуем вытащить с неё время публикации и id каждого поста. 

In [ ]:
[[d['id'],d['date']] for d in dw][:5]

Если кто-то не знает, вторая очень странная цифра в этом векторе — это время поста. А именно, количество секунд, которое прошло с $1970$ года до нашего дня. Чтобы перевести эту дату из такого формата в нормальный вид, можно воспользоваться пакетом `datetime`.

С помощью этого же пакета можно вычитать и складывать время. Чуть ниже, когда мы будем заворачивать весь придуманный нами код в функцию, мы добавим в неё аргумент, отвечающий за то, насколько древнюю информацию нужно скачивать. Будем делать это за последний год. 

In [ ]:
import datetime

# Это сегодняшний день
begin_data = datetime.date.today()
print(begin_data)

# Это день ровно год назад
end_data = begin_data.replace(year=begin_data.year - 1)
print(end_data)

Проходимся циклом по всем юзерам и запоминаем грязные стены. 

In [ ]:
dwd = { }       # aka dirty wall dict
for friend in tqdm_notebook(friends_ids):
    time.sleep(0.4) # не забываем дать серверу отдохнуть от нас
    try: 
        dw = Get_Wall(friend)
        dwd[friend] = [[d['id'],datetime.datetime.fromtimestamp(d['date'])] for d in dw]
    except:
        print('Возникли проблемы с', friend)

Первая цифра в нашем векторе — это `id` поста. Чтобы собрать данные о конкретной записи, можно использовать метод `likes.getList`.

In [ ]:
vk_download('likes.getList','type=post&owner_id='+str(friends_ids[0])+'&item_id='+str(dwd[friends_ids[0]]))

В качестве ответа получаем число лайков и список людей, которые лайкнули всё это чудо. Соберём данные по всей стене и сохраним их в виде cловарика. 

````{дата создания поста: [id лайкнувших] } ````

Мы могли бы взять и пройтись по всем юзерам и по всем id из нашего списка и получить все списки из лайкнувших. Однако, делать это напрямую слишком долго. К счастью, процесс можно ускорить с помощью метода [execute.](https://vk.com/dev/execute) Этот метод позволяет делать цепи из параллельных запросов к вк. Максимально в один батч из запросов можно поместить 25 запросов. Вы уже читали о нём в туториале и сейчас пришло время использовать его в боевых условиях. 

Напомню, что метод работает следующим образом. Мы должны перечислить через запятую внутри нашей ссылки все 25 запросов, которые мы делаем. И тогда в качестве ответа мы получим вектор из лайков.

In [ ]:
url = 'https://api.vk.com/method/execute?code=return[\
API.likes.getList({"type":"post","owner_id":"22261760","item_id":"3468","count":"1000"}),\
API.likes.getList({"type":"post","owner_id":"22261760","item_id":"3486","count":"1000"})];&access_token='+token

res = requests.get(url)
res.json( )

Например, в данной ситуации, мы попросили его отдать там юзеров, которые лайкнули два конкретных поста и получили их. Аналогично можно было бы ускорить код, который мы писали выше. 

Итак, перейдём к делу. Напишем функцию, которая для каждого конкретного пользователя будет понимать сколько у него на стене постов и будет делать из них батчи по 25 штук. Будем делать это для глубины в два года. 

In [ ]:
def make_batch(this_uid, dwd, end_date):
    # Делаем массив вида [id, дата]
    # В условии отбираем все даты, которые младще 2 лет
    wall = [wl for wl in dwd[this_uid] if wl[1].date() > end_date ]
    # Делим стенку на батчи 
    batches = [wall[i:i+25] for i in range(0,len(wall),25)]
    return batches 

In [ ]:
[len(item) for item in make_batch(all_uids[570], dwd, date.today().replace(year = date.today().year - 2))]

Вроде бы всё работает. Теперь напишем функцию, которая для батча будет генерировать запрос в виде ссылки. 

In [ ]:
def make_url(this_uid, batch, token):
    begin = 'https://api.vk.com/method/execute?code=return['
    end = '];&access_token='+token
    s = ''
    for bt in batch:
        s += 'API.likes.getList({"type":"post","owner_id":"' + str(this_uid) + \
             '","item_id":"' + str(bt[0]) + '","count":"1000"}),'
    return begin + s[:-1] + end 

In [ ]:
url = make_url(all_uids[570], make_bathc(all_uids[570])[-1], token)
url

In [ ]:
res = requests.get(url)
itg = res.json( )['response']
print(len(itg))
itg[1]

Получаем за один запрос дофига ответов. Всё работает. Пишем итоговую функцию. 

In [ ]:
def get_likes(this_uid, dwd=dwd, end_date = date.today().replace(year = date.today().year - 2), token = token):
    # Создаём под лайки вектор 
    wall_likes_u = { }
    # Делаем батчи
    batch = make_batch(this_uid, dwd, end_date)
    for btc in batch:
        # Создаём для батча ссылку 
        url = make_url(this_uid, btc, token)
        # Делаем запрос
        time.sleep(0.4)
        res = requests.get(url)
        itg = res.json( )['response']
        # Распаковываем в формате {время поста: юзеры, которые лайкали}
        wall_likes_u.update({datetime.datetime.fromtimestamp(b[1]).strftime('%Y-%m-%d %H:%M:%S') 
                             : it['users'] for b,it in zip(btc,itg)})
    return wall_likes_u

In [ ]:
loises = get_likes(all_uids[570])
loises['2016-01-20 06:50:47']

Ура! Всё работает. Теперь мы можем задействовать нашу функцию в промышленных масштабах. Пускаем цикл по всем пользователям. 

In [ ]:
wall_likes = { }
for uid in tqdm_notebook(all_uids):
    try:
        wall_likes[uid] = get_likes(uid)
    except:
        print('Возникли проблемы с ', uid)

In [ ]:
len(wall_likes)

Время работы кода в разы уменьшилось. Для того, чтобы по аналогии сделать с фотками, достаточно внести в код минимальный правки. Не будем делать это.

Нужно не забывать, что он можетбыть хитрым. Вполне возможно, что он скрыл из списка своих друзей всех тех (ну вы поняли), чтобы конкретно ТЫ, наша милая вооброжалка, не нашла их. Чтобы переиграть его, можно собрать всех друзей его друзей и поискать его лайки там.  Однако при таком подходе придётся собирать тысячи стен. Нужно будет придумать какой-то способ немного отфильтровать людей. 

https://python-graph-gallery.com/231-chord-diagram-with-bokeh/

https://habrahabr.ru/post/243229/

https://habrahabr.ru/post/221251/

Для чего это можно всё использовать? Конечно же для рисёрча! Можно собирать кучу разных данных по всему уютному контактику и что-то по ним изучать. 


## Почиташки 

* [Сказ о том как пацаны код ускоряли.](  https://habrahabr.ru/post/243229/)
* [Граф в d3,](https://habrahabr.ru/post/221251/) на основе которого делалась визуализация]


<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/cats.jpg" height="1200" width="1200"> 
